In [5]:
%matplotlib inline

from matplotlib import pyplot as plt
import itertools
import gym
import numpy as np
import tensorflow as tf
import datetime

seed = 0
np.random.seed(seed)
tf.random.set_seed(seed)

action_dim = env.action_space.n

logdir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [6]:
env = gym.make('BreakoutNoFrameskip-v4')
env = gym.wrappers.RecordVideo(env, './videos', name_prefix='breakout-v4')
# env = gym.wrappers.Monitor(env, directory='./videos/Breakout-v4', force=True)
observation = env.reset()

c:\Users\37103\AppData\Local\Programs\Python\Python39\lib\site-packages\gym\wrappers\record_video.py:41: UserWarning: WARN: Overwriting existing videos at d:\vscodespace\violet712.github.io\paper_notes100\DRL-1#Nature_DQN\videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


In [7]:
# while True:
#     # env.render(mode='rgb_array')
#     plt.figure()
#     plt.imshow(env.render(mode='rgb_array'), obs_type='grayscale')
#     action = env.action_space.sample()
#     observation, reward, done, info = env.step(action)
#     if done:
#         break
# env.close()


In [8]:
# for i in itertools.count():
#     action = env.action_space.sample()
#     observation, reward, done, info = env.step(action)
#     if i == 500:
#         break
# env.close()   

In [9]:
class QNetwork(tf.keras.Model):
    
    def __init__(self, name):
        super(QNetwork, self).__init__(name=name)
        self.conv1 = tf.keras.layers.Conv2D(
            32, kernel_size=(8, 8), strides=(4, 4),
            padding='valid', activation='relu')
        self.conv2 = tf.keras.layers.Conv2D(
            64, kernel_size=(4, 4), strides=(2, 2),
            padding='valid', activation='relu')
        self.conv3 = tf.keras.layers.Conv2D(
            64, kernel_size=(3, 3), strides=(1, 1),
            padding='valid', activation='relu')
        self.flat = tf.keras.layers.Flatten()
        self.fc1 = tf.keras.layers.Dense(512, activation='relu')
        self.fc2 = tf.keras.layers.Dense(action_dim, activation='linear')

    def call(self, pixels, **kwargs):
        # scale observation
        pixels = tf.divide(tf.cast(pixels, tf.float32), tf.constant(255.0))
        # extract features by convolutional layers
        feature = self.flat(self.conv3(self.conv2(self.conv1(pixels))))
        # calculate q_value
        qvalue = self.fc2(self.fc1(feature))

        return qvalue
        